In [2]:
import pandas as pd

In [5]:
gamesdf = pd.read_csv("../resources/clean_games.csv")


In [6]:
detailsdf = pd.read_csv("../resources/clean_games_details.csv")


In [8]:
rankingdf = pd.read_csv("../resources/clean_ranking.csv")


In [9]:
playersdf = pd.read_csv("../resources/players.csv")

In [ ]:
gamesdf.to_csv("../resources/games.csv", index=False)
detailsdf.to_csv("../resources/game_details.csv", index=False)
rankingdf.to_csv("../resources/ranking.csv", index=False)
playersdf.to_csv("../resources/players.csv", index=False)